In [ ]:
import os
import sys

# The path of the current Python script.
_CURRENT_PATH = os.path.dirname(os.path.realpath(__file__))
_TOP_PATH     = os.path.join(_CURRENT_PATH, 'dsta_mvs')

if _TOP_PATH not in sys.path:
    sys.path.insert( 0, _TOP_PATH)
    for i, p in enumerate(sys.path):
        print(f'{i}: {p}')

from api.dataset_player import DatasetProxy

dataset_player = DatasetProxy("./dsta_mvs/dsta_mvs/sample_project/conf_camera_space_vis.py", "./images")

# print(dataset_player.dataset.dist_lab_tab.bf)
# print(dataset_player.dataset.filenames.keys())
print(dataset_player.dataset[i].keys())
for i in range(len(dataset_player.dataset)):
    entry = dataset_player.dataset[i]
    print(entry['inv_dist_idx'].shape, entry['imgs'].shape, entry['rig_rgb'].shape)

In [ ]:
import requests
response = requests.request("GET", "http://127.0.0.1:9233/transformations")
D = response.json()['transformations']
D